In [ ]:
import glob
import numpy as np
import os
import tqdm

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
matplotlib.style.use( '/Users/zhafen/repos/clean-bold/clean-bold.mplstyle' )

In [ ]:
import anvil

In [ ]:
import verdict

In [ ]:
CHUNK_SIZE = 16
CHUNKS_PER_REGION = 32
REGION_SIZE = CHUNK_SIZE * CHUNKS_PER_REGION

# Parameters

In [ ]:
image_norm = 'z'
norm_dim_value = 0

In [ ]:
world_dir = '/Users/zhafen/data/minecraft/saves/aternos/world'

In [ ]:
color_fp = '/Users/zhafen/data/minecraft/textures/block/colors.h5'

# Load Data

In [ ]:
block_colors = verdict.Dict.from_hdf5( color_fp )

In [ ]:
region_fps = glob.glob( os.path.join( world_dir, 'region', '*' ) )

In [ ]:
i = 0
region_fp = region_fps[i]

In [ ]:
# Set up variables depending on what dimension is projected
if image_norm == 'z':
    xdim = 'x'
    ydim = 'y'
elif image_norm == 'y':
    xdim = 'z'
    ydim = 'x'
elif image_norm == 'x':
    xdim = 'z'
    ydim = 'y'
else:
    raise KeyError( 'Unrecognized image_norm, image_norm = {}'.format( image_norm ) )

In [ ]:
# Get region coords
_, x_region, z_region, _ = os.path.basename( region_fp ).split( '.' )
x_region, z_region = int( x_region ), int( z_region )

In [ ]:
# Make anvil classes
region = anvil.Region.from_file( region_fp )
chunk_origin = anvil.Chunk.from_region( region, 0, 0 )

In [ ]:
# Get the possible y values
section_height_range = anvil.chunk._section_height_range( chunk_origin.version )
y_range = np.arange( section_height_range.start * CHUNK_SIZE, section_height_range.stop * CHUNK_SIZE )

In [ ]:
# Convert indices to xs, ys, and zs, using index = y * 16**2 + z * 16 + x
inds = np.arange( 16**2 * y_range.size )

y_inds = inds // 16**2
z_inds = ( inds - y_inds * 16**2 ) // 16
x_inds = ( inds - y_inds * 16**2 - z_inds * 16 )

In [ ]:
# Extract data in string format
data_shape = ( REGION_SIZE, y_range.size )
data = np.zeros( data_shape, dtype='<U22' ).astype( str )
for i_chunk in tqdm.tqdm( range( CHUNKS_PER_REGION ) ):
    for k_chunk in range( CHUNKS_PER_REGION ):
        
            if k_chunk != norm_dim_value:
                continue
        
            # You can also provide the region file name instead of the object
            try:
                chunk = anvil.Chunk.from_region( region, i_chunk, k_chunk )
            except anvil.chunk.ChunkNotFound as e:
                missing_chunks.append( ( region_fp, i_chunk, k_chunk ) )
                continue
                
            for index, block in enumerate( chunk.stream_chunk() ):
                
                if z_inds[index] != 0:
                    continue
                                    
                data[i_chunk*16 + x_inds[index],y_inds[index]] = block.id

# Generate Map

In [ ]:
data_palette = np.unique( data )
img = np.ones( ( data.shape[0], data.shape[1], 4 ) )
for block_id in tqdm.tqdm( data_palette ):
    
    block_id_split = block_id.split( '_' )
    
    if block_id_split[0] == 'tall':
        block_id = '_'.join( block_id_split[1:] )
    
    try:
        block_color = block_colors[block_id]
        
    # Some have different colors for side or top
    except KeyError:
        if image_norm in [ 'x', 'z' ]:
            block_color = block_colors['{}_side'.format( block_id )]
        else:
            block_color = block_colors['{}_tope'.format( block_id )]
    
    # Color in the image
    img[np.where( data==block_id )] = block_color

In [ ]:
fig = plt.figure()
ax = plt.gca()

ax.imshow( np.rot90( img, axes=(0,1) ) )

ax.set_aspect( 'equal' )
ax.set_axis_off()